In [1]:
import pandas as pd
import requests
import numpy as np

# Insert your own client ID here
client_id = '5d87b00e-e162-40be-82fe-eb11fb0feb40'

In [2]:
endpoint= 'https://frost.met.no/sources/v0.jsonld?types=SensorSystem&country=NO'

r=requests.get(endpoint, auth=(client_id, ''))
json=r.json()
json

{'@context': 'https://frost.met.no/schema',
 '@type': 'SourceResponse',
 'apiVersion': 'v0',
 'license': 'https://creativecommons.org/licenses/by/3.0/no/',
 'createdAt': '2023-04-10T15:05:22Z',
 'queryTime': 1.63,
 'currentItemCount': 1339,
 'itemsPerPage': 1339,
 'offset': 0,
 'totalItemCount': 1339,
 'currentLink': 'https://frost.met.no/sources/v0.jsonld?types=SensorSystem&country=NO',
 'data': [{'@type': 'SensorSystem',
   'id': 'SN47230',
   'name': 'ÅKRA UNGDOMSSKOLE',
   'shortName': 'Åkra ',
   'country': 'Norge',
   'countryCode': 'NO',
   'geometry': {'@type': 'Point',
    'coordinates': [5.1963, 59.2555],
    'nearest': False},
   'masl': 18,
   'validFrom': '2013-10-29T00:00:00.000Z',
   'county': 'ROGALAND',
   'countyId': 11,
   'municipality': 'KARMØY',
   'municipalityId': 1149,
   'stationHolders': ['KARMØY KOMMUNE'],
   'externalIds': ['506131077'],
   'wigosId': '0-578-0-47230'},
  {'@type': 'SensorSystem',
   'id': 'SN23670',
   'name': 'E16 RYFOSS',
   'shortName': 

In [3]:
station_df=pd.DataFrame(json)

In [4]:
import json

dict_data = station_df['data']

# Normalize the dictionaries into a new DataFrame using json_normalize
new_df = pd.json_normalize(dict_data)

# Display the new DataFrame
print(new_df)


             @type       id               name    shortName country  \
0     SensorSystem  SN47230  ÅKRA UNGDOMSSKOLE        Åkra    Norge   
1     SensorSystem  SN23670         E16 RYFOSS   E16 Ryfoss   Norge   
2     SensorSystem  SN59450         STADLANDET   Stadlandet   Norge   
3     SensorSystem  SN12590        E6 MJØSBRUA  E6 Mjøsbrua   Norge   
4     SensorSystem  SN26640         E134 DARBU   E134 Darbu   Norge   
...            ...      ...                ...          ...     ...   
1334  SensorSystem  SN97120           E6 BARTA     E6 Barta   Norge   
1335  SensorSystem  SN61062         TOMREFJORD  Tomrefjord    Norge   
1336  SensorSystem  SN15270         JUVVASSHØE   Juvvasshøe   Norge   
1337  SensorSystem  SN74780        NAMSVATN II    Namsvatn    Norge   
1338  SensorSystem  SN94805        E69 VEDBOTN  E69 Vedbotn   Norge   

     countryCode    masl                 validFrom             county  \
0             NO    18.0  2013-10-29T00:00:00.000Z           ROGALAND   
1

In [5]:
import folium

In [6]:
# Split coordinates into separate columns
new_df['geometry.coordinates']=new_df['geometry.coordinates'].astype(str)
new_df[['longitude','latitude']] = new_df['geometry.coordinates'].str.strip('[]').str.split(',', expand=True).astype(float)


In [7]:
new_df.dropna(subset=['latitude', 'longitude'], inplace=True)

In [12]:
county_groups = {
    'AGDER': 'blue',
    'ROGALAND': 'blue',
    'OSLO': 'green',
    'VIKEN': 'green',
    'VESTFOLD OG TELEMARK': 'green',
    'INNLANDET': 'green',
    'TRØNDELAG': 'orange',
    'NORDLAND': 'purple',
    'TROMS OG FINNMARK': 'purple',
    'VESTLAND': 'red',
    'MØRE OG ROMSDAL': 'pink'
}

# Create map centered on Norway
norway_map = folium.Map(location=[60.4720, 8.4689], zoom_start=5, tiles="CartoDB positron")

# Add markers for each weather station in the selected counties
for index, row in new_df.iterrows():
    if row['county'] in county_groups:
        folium.CircleMarker(location=[row['latitude'], row['longitude']], radius=0.5, color=county_groups[row['county']], fill=True, fill_color=county_groups[row['county']], fill_opacity=0.4, icon=None).add_to(norway_map)

In [13]:
norway_map